## Import required libraries

In [200]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import yfinance as yf
%matplotlib inline
import warnings

In [201]:
# ignore unwanted warnings
warnings.filterwarnings('ignore')

## Data sources

### Load ticker data

In [202]:
def generate_ticker_returns(ticker):
    """Pulls financial data and computes yearly returns for a given ticker
    
    Args:
        ticker (str): the ticker symbol to download the financial data for
        
    Returns:
        dataframe: a pandas dataframe with yearly returns for the provided ticker
    """
    # download data sources from yfinance that covers the period for which we have ESG data
    df = yf.Ticker(ticker).history(start="2009-01-01", end="2023-01-01")
    # reset the index to access the Date column
    df.reset_index(inplace=True)
    # create a Year column from the Date column
    df['Year'] = df['Date'].dt.strftime('%Y')
    # convert the Year column to int64
    df['Year'] = df['Year'].astype('int')
    # keep only the Year and Close columns
    df = df[['Year','Close']]
    # pull the last occurance of each year to obtain a yearly breakdown of the prices
    df = df.groupby('Year').last()
    # compute the returns for the ticker
    df = df.pct_change()
    # clean up the initial NaN from the pct_change() call
    df.dropna(inplace=True)
    # rename the column header
    df.columns = ['Returns']
    
    # return the dataframe of yearly ticker returns
    return df

In [203]:
# generate ticker returns dataframes for target companies
tickers = ['AAPL', 'MSFT', 'CSCO', 'GOOG', 'INTC', 'NVDA', 'ORCL', 'TXN', '^GSPC']
tickers_dfs = list(map(generate_ticker_returns,tickers))

In [204]:
def create_alpha(ticker_df,market_df):
    """Nets out the corresponding market returns from those of the ticker
    
    Args:
        ticker_df (dataframe): dataframe of the ticker returns
        market_df (dataframe): dataframe of the sp500 returns
        
    Returns:
        dataframe: the ticker's returns dataframe with an added column for the alpha
    """
    # create the alpha column
    ticker_df['Alpha'] = ticker_df['Returns'] - market_df['Returns']
    
    # return the dataframe with the alpha column
    return ticker_df

In [205]:
# compute excess returns (returns above those of the S&P 500), also known as alpha, for each ticker dataframe
for ticker_df in tickers_dfs:
     ticker_df = create_alpha(ticker_df,tickers_dfs[-1])

# view a couple samples
display(tickers_dfs[0]) # apple_df
display(tickers_dfs[-1]) # sp500_df

,Returns,Alpha
Year,,
2010,0.530679,0.402852
2011,0.255580,0.255612
2012,0.325669,0.191612
2013,0.080695,-0.215318
2014,0.406225,0.292319
2015,-0.030137,-0.022871
2016,0.124804,0.029454
2017,0.484643,0.290443
2018,-0.053902,0.008471


,Returns,Alpha
Year,,
2010,0.127827,0.0
2011,-0.000032,0.0
2012,0.134057,0.0
2013,0.296012,0.0
2014,0.113906,0.0
2015,-0.007266,0.0
2016,0.095350,0.0
2017,0.194200,0.0
2018,-0.062373,0.0


### Load ESG scores

In [206]:
# load the ESG scores into a dataframe
ESG_data = Path("Sources/ESG Scores.csv")
ESG_df = pd.read_csv(ESG_data, index_col="Fiscal Year End Date",infer_datetime_format=True, parse_dates=True)
# view a sample
ESG_df.head()

,Organization ID,Fiscal Year,Special Value Flag,Feed Date,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,...,Asset4 Identifier,Organization ID.1,Refinitiv Business Classification Code,Primary Exchange,Status Flag (1 = Active),Organization Authority Identifier,Ticker,CUSIP,Sedol,ISIN
Fiscal Year End Date,,,,,,,,,,,,,,,,,,,,,
2010-12-25,18227,2010,500,9/25/2022,0.951624,0.515286,0.078947,0.953333,0.992754,0.833333,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2011-12-31,18227,2011,500,9/25/2022,0.905026,0.640013,0.375000,0.980263,0.979452,0.776923,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2012-12-29,18227,2012,500,9/25/2022,0.913084,0.913084,0.958333,0.981250,0.993333,0.778571,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2013-12-28,18227,2013,500,9/25/2022,0.856952,0.653476,0.450000,0.981250,0.993243,0.787671,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001
2014-12-27,18227,2014,500,9/25/2022,0.913875,0.648114,0.382353,0.981707,0.993421,0.783784,...,A4_00015,Intel Corporation,5710,NSM,1,4295906830,INTC,45814010,246324,US4581401001


In [222]:
# reset the index and only maintain relevant columns
ESG_df.reset_index(inplace=True)
ESG_df = ESG_df[['Fiscal Year','Ticker','ESG SCORE','ESG Combined Score','ESG Controversies Score','Resource Use Score','Emissions Score',
                 'Environmental Innovation Score','Workforce Score','Human Rights Score','Community Score','Product Responsibility Score','Management Score',
                 'Shareholders Score','CSR Strategy Score']]
# change the Fiscal Year column to Year
ESG_df.rename({'Fiscal Year':'Year'},axis=1,inplace=True)
# view a sample
ESG_df.head()

,Year,Ticker,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score
0,2010,INTC,0.951624,0.515286,0.078947,0.953333,0.992754,0.833333,0.994624,0.924242,0.972527,0.994048,0.980558,0.931705,0.974308
1,2011,INTC,0.905026,0.640013,0.375000,0.980263,0.979452,0.776923,0.972826,0.950000,0.959770,0.946429,0.825101,0.723286,0.976371
2,2012,INTC,0.913084,0.913084,0.958333,0.981250,0.993333,0.778571,0.983871,0.952830,0.927778,0.911765,0.909694,0.651531,0.969203
3,2013,INTC,0.856952,0.653476,0.450000,0.981250,0.993243,0.787671,0.983696,0.983607,0.927778,0.889535,0.554248,0.679120,0.970217
4,2014,INTC,0.913875,0.648114,0.382353,0.981707,0.993421,0.783784,0.973118,0.984615,0.907609,0.882022,0.921699,0.622825,0.968007


### Combine ticker returns and ESG data in the same dataframe

In [223]:
def combined_ticker_and_ESG_data(ticker_df,ticker):
    """Combines the ticker's returns and corresponding ESG data into a dataframe
    
    Args:
        ticker_df (dataframe): dataframe of the ticker returns
        ticker (str): the ticker symbol to pull ESG data for
        
    Returns:
        dataframe: the ticker's returns combined with its corresponding ESG data
    """
    # generate a dataframe for the ticker's ESG data
    ticker_esg_data_df = ESG_df.query('Ticker == @ticker')
    # turn the Year column to an index
    ticker_esg_data_df.set_index('Year',inplace=True) # this throws a 'SettingWithCopyWarning' warning that we purposely ignore
    # combine the two dataframes
    ticker_df = pd.concat([ticker_df,ticker_esg_data_df], axis="columns", join="inner")
    
    # return the dataframe with the combined returns and ESG data for the ticker
    return ticker_df

In [211]:
# combine ticker returns dataframes with their corresponding ESG data
combined_tickers_dfs = list(map(combined_ticker_and_ESG_data,tickers_dfs[:-1],tickers[:-1]))

# create an individual dataframe for each ticker
apple_df = combined_tickers_dfs[0]
msft_df = combined_tickers_dfs[1]
cisco_df = combined_tickers_dfs[2]
google_df = combined_tickers_dfs[3]
intel_df = combined_tickers_dfs[4]
nvidia_df = combined_tickers_dfs[5]
oracle_df = combined_tickers_dfs[6]
texinst_df = combined_tickers_dfs[7]
sp500_df = tickers_dfs[8]
# view a couple samples
display(apple_df)
display(nvidia_df)
display(sp500_df)

,Returns,Alpha,Ticker,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score
Year,,,,,,,,,,,,,,,,
2010,0.530679,0.402852,AAPL,0.598853,0.313315,0.027778,0.937500,0.587500,0.400000,0.467391,0.948276,0.744444,0.050000,0.572782,0.912762,0.595850
2011,0.255580,0.255612,AAPL,0.526378,0.277078,0.027778,0.937500,0.462500,0.432432,0.436170,0.954545,0.931818,0.048780,0.085181,0.858367,0.809074
2012,0.325669,0.191612,AAPL,0.565545,0.298397,0.031250,0.858974,0.448718,0.418919,0.366667,0.944444,0.837209,0.040541,0.399490,0.984184,0.772645
2013,0.080695,-0.215318,AAPL,0.602998,0.319356,0.035714,0.833333,0.423077,0.388889,0.388889,0.945946,0.642857,0.039474,0.728250,0.966735,0.765343
2014,0.406225,0.292319,AAPL,0.561743,0.297538,0.033333,0.807692,0.581081,0.385714,0.476744,0.585714,0.550000,0.041667,0.829580,0.923746,0.778793
2015,-0.030137,-0.022871,AAPL,0.542147,0.298851,0.055556,0.922222,0.621951,0.368421,0.578431,0.463415,0.549020,0.133333,0.719056,0.901042,0.769544
2016,0.124804,0.029454,AAPL,0.615581,0.375972,0.136364,0.904255,0.616279,0.392857,0.640351,0.455556,0.500000,0.555556,0.854309,0.884172,0.762481
2017,0.484643,0.290443,AAPL,0.683317,0.409840,0.136364,0.970000,0.670213,0.400000,0.867188,0.522222,0.671875,0.616667,0.866472,0.887041,0.762759
2018,-0.053902,0.008471,AAPL,0.718954,0.380310,0.041667,0.991525,0.728070,0.400000,0.750000,0.613208,0.878571,0.590909,0.940266,0.846423,0.755582


,Returns,Alpha,Ticker,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score
Year,,,,,,,,,,,,,,,,
2010,-0.175589,-0.303416,NVDA,0.606981,0.316649,0.026316,0.860000,0.659420,0.416667,0.876344,0.106061,0.884615,0.000000,0.931705,0.892822,0.683794
2011,-0.100000,-0.099968,NVDA,0.570155,0.570155,1.000000,0.717105,0.705479,0.392308,0.853261,0.100000,0.902299,0.000000,0.781754,0.990423,0.624764
2012,-0.109733,-0.243790,NVDA,0.634779,0.634779,1.000000,0.781250,0.793333,0.342857,0.865591,0.103774,0.905556,0.170588,0.970918,0.964796,0.851449
2013,0.335238,0.039225,NVDA,0.758774,0.758774,1.000000,0.856250,0.844595,0.349315,0.798913,0.795082,0.983333,0.168605,0.999488,0.940123,0.847473
2014,0.273958,0.160051,NVDA,0.782896,0.782896,1.000000,0.932927,0.875000,0.358108,0.876344,0.669231,0.777174,0.769663,0.996418,0.761003,0.859232
2015,0.671155,0.678421,NVDA,0.744623,0.744623,1.000000,0.970930,0.930380,0.347222,0.785000,0.371622,0.925000,0.781915,0.996630,0.685968,0.848534
2016,2.269604,2.174253,NVDA,0.733099,0.733099,1.000000,0.984694,0.929348,0.328947,0.856522,0.311828,0.917391,0.754545,0.992534,0.597056,0.849470
2017,0.819879,0.625679,NVDA,0.765966,0.765966,1.000000,0.940367,0.964646,0.350000,0.921642,0.459596,0.929104,0.776000,0.995029,0.568564,0.844828
2018,-0.308152,-0.245779,NVDA,0.757605,0.757605,0.857143,0.946281,0.978261,0.346154,0.954225,0.357759,0.926056,0.788321,0.997837,0.600166,0.843126


,Returns,Alpha,Ticker,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score
Year,,,,,,,,,,,,,,,,
2010,-0.041953,-0.169780,GOOG,0.436202,0.225454,0.014706,0.459770,0.753012,0.718750,0.760714,0.000000,0.582143,0.247788,0.536889,0.040379,0.119565
2011,0.087429,0.087460,GOOG,0.475564,0.290813,0.106061,0.408046,0.844828,0.701493,0.802158,0.000000,0.938406,0.228070,0.494456,0.016633,0.114367
2012,0.095185,-0.038872,GOOG,0.492676,0.253914,0.015152,0.576531,0.845745,0.661765,0.795139,0.000000,0.830986,0.237500,0.588265,0.012755,0.100543
2013,0.584311,0.288299,GOOG,0.435804,0.228772,0.021739,0.664948,0.835106,0.632812,0.813725,0.000000,0.453333,0.229508,0.584954,0.012794,0.101083
2014,-0.059656,-0.173562,GOOG,0.518395,0.267531,0.016667,0.788660,0.828125,0.606557,0.803030,0.000000,0.687117,0.257353,0.719038,0.007677,0.090494
2015,0.445599,0.452865,GOOG,0.621599,0.321669,0.021739,0.829167,0.926724,0.600000,0.903689,0.000000,0.672131,0.329384,0.925551,0.037684,0.404723
2016,0.017051,-0.078299,GOOG,0.622739,0.319434,0.016129,0.931655,0.865385,0.589041,0.926710,0.608911,0.674267,0.347985,0.768985,0.103456,0.397882
2017,0.355757,0.161557,GOOG,0.635047,0.323476,0.011905,0.950658,0.836806,0.545455,0.974531,0.583969,0.920912,0.356037,0.708433,0.019369,0.409655
2018,-0.010312,0.052061,GOOG,0.663089,0.345181,0.027273,0.940415,0.855191,0.531250,0.979810,0.630814,0.909739,0.937500,0.528952,0.099334,0.563455


,Returns,Alpha
Year,,
2010,0.127827,0.0
2011,-0.000032,0.0
2012,0.134057,0.0
2013,0.296012,0.0
2014,0.113906,0.0
2015,-0.007266,0.0
2016,0.095350,0.0
2017,0.194200,0.0
2018,-0.062373,0.0


### Combine additional dataframes for analysis

In [216]:
# create a stock returns performance dataframe
stock_returns_performance_df = pd.concat([apple_df['Returns'],msft_df['Returns'],cisco_df['Returns'],google_df['Returns'],intel_df['Returns'],nvidia_df['Returns'],oracle_df['Returns'],texinst_df['Returns'],sp500_df['Returns']],axis=1,join='inner')
# set column names 
stock_returns_performance_df.columns = ['APPLE','MSFT','CISCO','GOOGLE','INTEL','NVIDIA','ORACLE','TEX INS','SP500']
# clean out any nulls and remove duplicates; which there shouldn't be any
stock_returns_performance_df = stock_returns_performance_df.dropna().drop_duplicates()
# view a sample
stock_returns_performance_df.head()

,APPLE,MSFT,CISCO,GOOGLE,INTEL,NVIDIA,ORACLE,TEX INS,SP500
Year,,,,,,,,,
2010,0.530679,-0.065246,-0.154971,-0.041953,0.062695,-0.175589,0.286125,0.270755,0.127827
2011,0.255580,-0.045156,-0.096172,0.087429,0.193964,-0.100000,-0.174621,-0.088761,-0.000032
2012,0.325669,0.057988,0.116056,0.095185,-0.120146,-0.109733,0.317170,0.087131,0.134057
2013,0.080695,0.442980,0.167266,0.584311,0.309200,0.335238,0.156824,0.462866,0.296012
2014,0.406225,0.275646,0.279759,-0.059656,0.442240,0.273958,0.189748,0.250923,0.113906


In [217]:
# create a stock alpha performance dataframe
stock_alpha_performance_df = pd.concat([apple_df['Alpha'],msft_df['Alpha'],cisco_df['Alpha'],google_df['Alpha'],intel_df['Alpha'],nvidia_df['Alpha'],oracle_df['Alpha'],texinst_df['Alpha']],axis=1,join='inner')
# set column names
stock_alpha_performance_df.columns = ['APPLE','MSFT','CISCO','GOOGLE','INTEL','NVIDIA','ORACLE','TEX INS']
# clean out any nulls and remove duplicates; which there shouldn't be any
stock_alpha_performance_df = stock_alpha_performance_df.dropna().drop_duplicates()
# view a sample
stock_alpha_performance_df.head()

,APPLE,MSFT,CISCO,GOOGLE,INTEL,NVIDIA,ORACLE,TEX INS
Year,,,,,,,,
2010,0.402852,-0.193073,-0.282798,-0.169780,-0.065132,-0.303416,0.158298,0.142928
2011,0.255612,-0.045125,-0.096140,0.087460,0.193996,-0.099968,-0.174589,-0.088729
2012,0.191612,-0.076069,-0.018001,-0.038872,-0.254203,-0.243790,0.183113,-0.046926
2013,-0.215318,0.146967,-0.128747,0.288299,0.013188,0.039225,-0.139189,0.166854
2014,0.292319,0.161740,0.165852,-0.173562,0.328334,0.160051,0.075842,0.137017


## Analysis

### Data description and distribution - ticker returns

In [220]:
# observed the description of stock returns
stock_returns_performance_df.describe()

,APPLE,MSFT,CISCO,GOOGLE,INTEL,NVIDIA,ORACLE,TEX INS,SP500
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,0.301488,0.223432,0.098778,0.179980,0.081437,0.493917,0.126604,0.198994,0.109042
std,0.336120,0.255341,0.194608,0.291992,0.242093,0.782756,0.182242,0.206112,0.144637
min,-0.264042,-0.280247,-0.224627,-0.386713,-0.466427,-0.502646,-0.176311,-0.098574,-0.194428
25%,0.080695,0.057988,-0.034865,-0.010312,-0.022139,-0.109733,-0.029696,0.051743,-0.000032
50%,0.325669,0.226919,0.138126,0.095185,0.062695,0.335238,0.189748,0.250923,0.127827
75%,0.484643,0.425341,0.167266,0.355757,0.307120,0.819879,0.249368,0.367113,0.194200
max,0.889578,0.575581,0.457613,0.651706,0.442240,2.269604,0.368895,0.467459,0.296012


In [ ]:
# plot a bar chart of the stock returns


In [219]:
# observed the description of stock alphas
stock_alpha_performance_df.describe()

,APPLE,MSFT,CISCO,GOOGLE,INTEL,NVIDIA,ORACLE,TEX INS
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,0.192445,0.114389,-0.010264,0.070937,-0.027606,0.384874,0.017561,0.089951
std,0.259577,0.164132,0.158070,0.210386,0.192014,0.725998,0.126142,0.121533
min,-0.215318,-0.193073,-0.282798,-0.192285,-0.309477,-0.308218,-0.174589,-0.093618
25%,0.008471,-0.045125,-0.128747,-0.078299,-0.208402,-0.243790,-0.095348,-0.009322
50%,0.191612,0.161740,-0.018001,0.052061,-0.007786,0.160051,0.055169,0.106854
75%,0.292319,0.255842,0.118483,0.161557,0.104638,0.678421,0.099967,0.154424
max,0.660478,0.286800,0.228089,0.452865,0.328334,2.174253,0.183113,0.273259


In [ ]:
# plot a bar chart of the stock alphas


### Data description and distribution - ESG data

In [221]:
# observed the description of ESG data
ESG_df.describe()

,Fiscal Year,ESG SCORE,ESG Combined Score,ESG Controversies Score,Resource Use Score,Emissions Score,Environmental Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score
count,104.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,2016.000000,0.755714,0.558715,0.402963,0.931577,0.892858,0.586839,0.882461,0.686444,0.894162,0.636796,0.781256,0.606406,0.744611
std,3.759777,0.152577,0.201458,0.383672,0.104438,0.131547,0.206265,0.133715,0.325744,0.119383,0.326234,0.250125,0.271437,0.244802
min,2010.000000,0.435804,0.225454,0.005000,0.408046,0.423077,0.323529,0.366667,0.000000,0.453333,0.000000,0.085181,0.007677,0.075643
25%,2013.000000,0.606981,0.395241,0.055556,0.911765,0.855191,0.400000,0.846535,0.583969,0.880208,0.347985,0.623414,0.435910,0.599912
50%,2016.000000,0.806383,0.526480,0.258065,0.974820,0.952055,0.542553,0.926710,0.834821,0.943478,0.776000,0.891698,0.645424,0.843126
75%,2019.000000,0.882782,0.751087,0.812500,0.991279,0.973404,0.718750,0.972826,0.933962,0.964286,0.932306,0.982088,0.824533,0.923218
max,2022.000000,0.951624,0.925424,1.000000,0.998927,0.996528,0.993750,0.997642,0.989899,0.997951,0.994048,0.999488,0.990423,0.976371


In [ ]:
# plot charts to display relationships in the ESG data


### Ticker to ESG data correlation